In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nasreddin/joke_tokenizer.model
/kaggle/input/nasreddin/nasreddin_hoca_jokes.json
/kaggle/input/nasreddin/cleaned_nasreddin_hoca.txt


In [10]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset
import pandas as pd
import torch

# GPU kontrolü
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
import json
import pandas as pd
from datasets import Dataset

# JSON dosyasını oku (full path ile)
with open("/kaggle/input/nasreddin/nasreddin_hoca_jokes.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Pandas DataFrame'e çevir
df = pd.DataFrame(data)

# input + output'u birleştir
df["text"] = df["input"] + " " + df["output"]

# HF dataset'e dönüştür
dataset = Dataset.from_pandas(df[["text"]])


In [12]:
# 1. Veri(güncellemeye açık hücre)
df = pd.read_json("/kaggle/input/nasreddin/nasreddin_hoca_jokes.json")
df["text"] = df["input"] + " " + df["output"]
dataset = Dataset.from_pandas(df[["text"]])

# 2. Tokenizer ve model
model_name = "ytu-ce-cosmos/turkish-gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

# 3. Tokenize
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)
tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/225 [00:00<?, ? examples/s]

In [13]:

# 4. Eğitim argümanları

training_args = TrainingArguments(
    output_dir="./joke_model",
    per_device_train_batch_size=2,  # Daha fazla veri tutabiliyorsa 2 yap
    num_train_epochs=10,             # Daha fazla öğrenme
    learning_rate=1e-5,             # Daha yumuşak öğrenme
    warmup_steps=50,                # Isınma dönemi
    weight_decay=0.01,              # Regularization
    logging_dir="./logs",           # TensorBoard için
    logging_steps=10,
    save_steps=250,                 # Checkpoint aralığı artırıldı
    save_total_limit=2,     
    report_to="none",              # wandb vs istemiyorsun
)



# 5. Trainer
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# 6. Eğitimi başlat
trainer.train()

/tmp/ipykernel_31/2124091181.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,2.105000
20,2.001900
30,2.130800
40,2.002400
50,1.835100
60,2.160000
70,1.957000
80,1.712500
90,1.584600
100,1.688600


TrainOutput(global_step=570, training_loss=1.5653894876178942, metrics={'train_runtime': 83.4762, 'train_samples_per_second': 26.954, 'train_steps_per_second': 6.828, 'total_flos': 146976768000000.0, 'train_loss': 1.5653894876178942, 'epoch': 10.0})

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, pipeline

# 1. Fine-tuned model yolunu belirt
model_path = "/kaggle/working/joke_model/checkpoint-570"

# 2. Tokenizer'ı yükle
tokenizer = GPT2Tokenizer.from_pretrained("ytu-ce-cosmos/turkish-gpt2")

# 3. Fine-tuned modeli yükle
model = GPT2LMHeadModel.from_pretrained(model_path)

# 4. Üretim pipeline'ı oluştur
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# 5. Prompt (başlangıç ifadesi)
prompt = "Nasreddin Hoca'nın başına komik bir olay geldi. "


# 6. Şaka üret
outputs = generator(
    prompt,
    max_length=100,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.7,
    top_k=40,
    top_p=0.9,
    repetition_penalty=1.5,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    early_stopping=True,
)

# 7. Sonucu yazdır
print("🗣️ Üretilen Fıkra:\n")
print(outputs[0]["generated_text"].replace(prompt, "").strip())
